# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,enewetak,11.3474,162.3373,28.47,78,91,5.21,MH,1689885107
1,1,edinburgh of the seven seas,-37.0676,-12.3116,11.74,59,24,9.96,SH,1689885110
2,2,college,64.8569,-147.8028,19.18,90,100,1.54,US,1689885133
3,3,serafimovich,49.5786,42.7360,18.94,57,27,2.11,RU,1689885213
4,4,tazovsky,67.4667,78.7000,17.14,75,57,4.34,RU,1689885134


In [44]:
city_df.to_excel("cities_data.xlsx", index=False)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_df.hvplot.points('Lng', 'Lat', size='Humidity', color='City', geo=True, tiles= "OSM", scale=1
                                 ,alpha=0.5,frame_width=700, frame_height=500)
# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_df[
    (city_df['Max Temp'] > 21) & 
    (city_df['Max Temp'] < 27) & 
    (city_df['Wind Speed'] < 4.5) & 
    (city_df['Cloudiness'] == 0)]

# Display sample data
# YOUR CODE HERE
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,colonia,50.9333,6.9500,21.08,59,0,2.06,DE,1689885196
120,120,as-suwayda,32.7090,36.5695,23.63,25,0,1.97,SY,1689885221
127,127,fort mcmurray,56.7268,-111.3810,25.77,47,0,2.57,CA,1689885222
146,146,urzhar,47.0930,81.6294,21.73,49,0,2.37,KZ,1689885015
181,181,kaspiysk,42.8816,47.6392,22.10,64,0,4.02,RU,1689885226


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_weather_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,colonia,DE,50.9333,6.9500,59,
120,as-suwayda,SY,32.7090,36.5695,25,
127,fort mcmurray,CA,56.7268,-111.3810,47,
146,urzhar,KZ,47.0930,81.6294,49,
181,kaspiysk,RU,42.8816,47.6392,64,
277,mizque,BO,-17.9410,-65.3402,37,
316,xacmaz,AZ,41.4635,48.8060,64,
319,podari,RO,44.2500,23.7833,71,
385,izazi,TZ,-7.2000,35.7333,42,
409,archangelos,GR,36.2149,28.1149,55,


In [16]:
url = f"https://api.geoapify.com/v1/geocode/reverse?lat={lat}&lon={lng}&limit=1&filter=amenity=hotels&apiKey={geoapify_key}" 
response = requests.get(url).json()
response

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright'},
    'country': 'Germany',
    'country_code': 'de',
    'state': 'North Rhine-Westphalia',
    'city': 'Cologne',
    'postcode': '50676',
    'district': 'Innenstadt',
    'neighbourhood': 'Cäcilienviertel',
    'suburb': 'Altstadt-Süd',
    'street': 'Agrippastraße',
    'housenumber': '98',
    'lon': 6.950295529924231,
    'lat': 50.93323545,
    'distance': 0,
    'result_type': 'building',
    'formatted': 'Agrippastraße 98, 50676 Cologne, Germany',
    'address_line1': 'Agrippastraße 98',
    'address_line2': '50676 Cologne, Germany',
    'category': 'building',
    'timezone': {'name': 'Europe/Berlin',
     'offset_STD': '+01:00',
     'offset_STD_seconds': 3600,
     'offset_DST': '+02:00',
     'offset_D

In [14]:
import requests
# Import the API key
from api_keys import geoapify_key
import pprint
# Function to find the nearest hotel using Geoapify API
def find_nearest_hotel(city, lat, lng):
    url = f"https://api.geoapify.com/v1/geocode/reverse?lat={lat}&lon={lng}&limit=1&filter=amenity=hotels&apiKey={geoapify_key}"
    response = requests.get(url).json()
    if 'features' in response:
        if len(response['features']) > 0:
            hotel_name = response['features'][0]['properties']['"address_line1']
            print(f"The nearest hotel in {city} is: {hotel_name}")
        else:
            print(f"No hotel found in {city}")
    else:
        print(f"Error retrieving data for {city}")

# Iterate over each row in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    
    # Find the nearest hotel for the current city
    find_nearest_hotel(city, lat, lng)


KeyError: '"address_line1'

In [17]:
import json

radius = 10000
limit = 1
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)
    name_address = response.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df


Starting hotel search
colonia - nearest hotel: Wasserturm Hotel Cologne
as-suwayda - nearest hotel: Dionysias Hotel
fort mcmurray - nearest hotel: Nomad Hotel and Suites
urzhar - nearest hotel: Sharikan
kaspiysk - nearest hotel: Sanatorium "Energetic"
mizque - nearest hotel: Hotel Victoria
xacmaz - nearest hotel: Xaçmaz Otel
podari - nearest hotel: Belvedere
izazi - nearest hotel: No hotel found
archangelos - nearest hotel: Semina
saint-gely-du-fesc - nearest hotel: Kyriad Montpellier Nord
birjand - nearest hotel: ITTIC Birjand
monteagudo - nearest hotel: Residencial San José
sona - nearest hotel: Al Bosco
saint-pierre - nearest hotel: Tropic Hotel
fort st. john - nearest hotel: Fort St. John Motor Inn
grasse - nearest hotel: Hôtel Napoléon
lepsy - nearest hotel: Orley


,City,Country,Lat,Lng,Humidity,Hotel Name
25,colonia,DE,50.9333,6.9500,59,Wasserturm Hotel Cologne
120,as-suwayda,SY,32.7090,36.5695,25,Dionysias Hotel
127,fort mcmurray,CA,56.7268,-111.3810,47,Nomad Hotel and Suites
146,urzhar,KZ,47.0930,81.6294,49,Sharikan
181,kaspiysk,RU,42.8816,47.6392,64,"Sanatorium ""Energetic"""
277,mizque,BO,-17.9410,-65.3402,37,Hotel Victoria
316,xacmaz,AZ,41.4635,48.8060,64,Xaçmaz Otel
319,podari,RO,44.2500,23.7833,71,Belvedere
385,izazi,TZ,-7.2000,35.7333,42,No hotel found
409,archangelos,GR,36.2149,28.1149,55,Semina


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    Hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng', 'Lat', size='Humidity', color='City', geo=True, tiles= "OSM", hover=True, hover_cols=['City', 'Country', 'Hotel Name'],
                                 alpha=0.5,frame_width=700, frame_height=500)
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)